In [1]:
#Importering av biblioteker
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display
import calendar

#Plotly
import plotly.express as px

#Bokeh
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

df = pd.read_csv('komplett_data_med_utfylte_mengder.csv',parse_dates=['datetime'])



In [2]:
df_2011 = df[df['datetime'].dt.year == 2011]
df_2012 = df[df['datetime'].dt.year == 2012]
df_2013 = df[df['datetime'].dt.year == 2013]
df_2014 = df[df['datetime'].dt.year == 2014]
df_2015 = df[df['datetime'].dt.year == 2015]
df_2016 = df[df['datetime'].dt.year == 2016]
df_2017 = df[df['datetime'].dt.year == 2017]
df_2018 = df[df['datetime'].dt.year == 2018]
df_2019 = df[df['datetime'].dt.year == 2019]
df_2020 = df[df['datetime'].dt.year == 2020]
df_2021 = df[df['datetime'].dt.year == 2021]

måneder_norsk = ['Januar', 'Februar', 'Mars', 'April', 'Mai', 'Juni',
                 'Juli', 'August', 'September', 'Oktober', 'November', 'Desember']
år_liste = list(range(2011, 2022)) 


In [24]:

måned_options = [(navn, i) for i, navn in enumerate(måneder_norsk, start=1)]

# Dropdown many for første periode
år_1 = widgets.Dropdown(options=år_liste, value=2011, description="År 1:")
måned_1 = widgets.Dropdown(options=måned_options, value=1, description="Måned 1:")

# Dropdown many for andre periode
år_2 = widgets.Dropdown(options=år_liste, value=2021, description="År 2:")
måned_2 = widgets.Dropdown(options=måned_options, value=1, description="Måned 2:")

# Dropbdown meny for visningsalternativer
visningstype = widgets.Dropdown(options=["Linjegraf", "Søylediagram", "Boksplott"],
    value="Linjegraf", description="Visning:")

# info boks om produsert MW
info_boks = widgets.HTML()
info_boks.layout = widgets.Layout(width='320px', padding='10px 20px')

# Funksjon for å hente og tilpasse data
def hent_data(år, måned):
    navn = måneder_norsk[måned - 1]
    data = df[(df["datetime"].dt.year == år) & (df["datetime"].dt.month == måned)].copy()
    data["visningstid"] = data["datetime"].apply(lambda x: x.replace(year=2025, month=1))
    data["label"] = f"{navn} {år}"
    return data

# Funksjon som lager plottet
def sammenlign_plot(år_1, måned_1, år_2, måned_2, visningstype):
    data_1 = hent_data(år_1, måned_1)
    data_2 = hent_data(år_2, måned_2)

    if data_1.empty or data_2.empty:
        info_boks.value = "<b>En eller begge valgte perioder har ingen tilgjengelig data.</b>"
        return

    samlet = pd.concat([data_1, data_2])

    # Velg graf basert på visningstype
    if visningstype == "Linjegraf":
        fig = px.line(
            samlet, x="visningstid", y="MW", color="label",
            title=f"Sammenligning av periode: {data_1['label'].iloc[0]} og {data_2['label'].iloc[0]}",
            labels={"visningstid": "Tid i måneden", "MW": "Megawatt (MW)"}
        )
        fig.update_xaxes(tickformat="%d")
        fig.update_layout(hovermode="x unified")

    elif visningstype == "Søylediagram":
        totals = samlet.groupby("label")["MW"].sum().reset_index()
        fig = px.bar(
            totals, x="label", y="MW", color="label",
            title="Total MW-produksjon i valgt måned",
            labels={"MW": "Megawatt (MW)", "label": "Periode"}
        )

    elif visningstype == "Boksplott":
        fig = px.box(
            samlet, x="label", y="MW", color="label",
            title="Fordeling av MW-produksjon per periode",
            labels={"MW": "Megawatt (MW)", "label": "Periode"}
        )

    fig.show()

    # Beregn totalsummer og vis i infoboks
    total_1 = int(data_1["MW"].sum())
    total_2 = int(data_2["MW"].sum())
    forskjell = total_2 - total_1
    tegn = "+" if forskjell >= 0 else ""

    info_boks.value = f"""
    <div style="border:1px solid #ccc; border-radius:8px; background-color:#f9f9f9; padding:10px 15px;">
      <h4 style='margin-bottom:6px;'>Totalt produsert MW i periodene:</h4>
      <ul style='margin-top:0'>
        <li><b>{data_1['label'].iloc[0]}</b>: {total_1:,} MW</li>
        <li><b>{data_2['label'].iloc[0]}</b>: {total_2:,} MW</li>
        <li><b>Forskjellen fra periode 1 til 2:</b> {tegn}{forskjell:,} MW</li>
      </ul>
    </div>
    """

# Layout for menyvalg og interaktiv oppdatering
venstre = widgets.VBox([år_1, måned_1, visningstype])
høyre = widgets.VBox([år_2, måned_2])
layout = widgets.HBox([venstre, høyre])
interaktiv = widgets.interactive_output(
    sammenlign_plot,
    {
        "år_1": år_1, "måned_1": måned_1,
        "år_2": år_2, "måned_2": måned_2,
        "visningstype": visningstype
    }
)
graf_og_info = widgets.HBox([interaktiv, info_boks])

# Vis alt
display(layout, graf_og_info)
